In [1]:
#Data source
dpath = "/home/ubuntu/data/"
import pandas as pd, numpy as np

#Preprocessing steps
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

#Metrics to use
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
beta = 2 #favors recall 
from sklearn.model_selection import cross_val_score

#Classifiers
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
from __future__ import print_function

In [3]:
#List of all classifiers that will be trained
classifiers = [LogisticRegression(n_jobs=-1), RandomForestClassifier(n_jobs=-1), GaussianNB(), 
               MLPClassifier(), KNeighborsClassifier(n_jobs=-1), SVC(probability=True)]

In [4]:
#List of all features that will be combined in late fusion
feat_to_combine = ["AutoColorCorrelogram", "BinaryPatternsPyramid", "EdgeHistogram", "FCTH", "FuzzyColorHistogram", 
                       "FuzzyOpponentHistogram", "Gabor", "JCD", "JpegCoefficientHistogram", "LocalBinaryPatterns", 
                       "LuminanceLayout", "OpponentHistogram", "PHOG", "RotationInvariantLocalBinaryPatterns", "ScalableColor", 
                       "SimpleColorHistogram", "Tamura", "CEDD", "ColorLayout"]

In [5]:
#Loading of the train and test set descriptors
train_set = pd.read_csv(dpath + "train_set.csv")
test_set = pd.read_csv(dpath + "test_set.csv")

In [7]:
#Creation of the feature set list

#We'll load all the feature files, preprocess the data and store it as tuples of train,test sets

feature_sets = {}
for feat in feat_to_combine:
    features_train = pd.read_csv(dpath + "features_%s_%s.csv" % (feat, "train"), header=None, skiprows=1)
    features_train = features_train.drop(features_train.columns[0], axis=1)
    features_test = pd.read_csv(dpath + "features_%s_%s.csv" % (feat, "test"), header=None, skiprows=1) 
    features_test = features_test.drop(features_test.columns[0], axis=1)
    
    #Feature transformation pipeline: only MinMaxScaling but more can be added later
    processing_pipeline = make_pipeline(MinMaxScaler())
    X_features_train = processing_pipeline.fit_transform(features_train.as_matrix())
    X_features_test = processing_pipeline.transform(features_test.as_matrix())
    
    feature_sets[feat] = (X_features_train, X_features_test)


In [8]:
#Function to obtain the classifier name from their string representation
def classifier_name(classifier):
    return str(classifier).split("(")[0]

In [9]:
#ids and targets

id_train = train_set["Image"]
y_train = train_set["Class"]

id_test = test_set["Image"]
y_test = test_set["Class"]

In [10]:
def train_and_predict(classifier, X_features_train, y_train, X_features_test):
    
    #Obtaining cross-validated metrics
    score = cross_val_score(classifier, X_features_train, y_train, scoring="accuracy", cv=4, n_jobs=4).mean()

    #Training with the full train set and making predictions on the test set
    classifier = classifier.fit(X_features_train, y_train)
    prediction = classifier.predict_proba(X_features_test)
    
    return prediction, score

In [12]:
%%time

metrics = pd.DataFrame(columns=["model", "accuracy", "F1" ,"precision", "recall", "Fbeta"])
model_scores = {}
model_predictions = {}

for classifier in classifiers: 
    
    model_name = classifier_name(classifier)
    print("--- Model:", model_name)
    
    l1_scores = {}
    l1_test_predictions = {}
    
    for feat_set in feat_to_combine:
        print("- features:", feat_set)
        features_train, features_test = feature_sets[feat_set]
        
        prediction, score = train_and_predict(classifier, features_train, y_train, features_test)
        
        l1_scores[feat_set] = score
        l1_test_predictions[feat_set] = prediction
        
    
    model_scores[model_name] = l1_scores
    l1_test_predictions = pd.Panel(l1_test_predictions)
    
    #To combine predictions, we can average the probability predictions and choose the highest
    prob_ave = l1_test_predictions.mean(axis=0).as_matrix()
    pred_classes_ave = [classifier.classes_[i] for i in np.argmax(prob_ave, axis=1)]
    
    m_ave = {"model": model_name+"_Average", 
         "accuracy": accuracy_score(y_test, pred_classes_ave), 
         "F1": f1_score(y_test, pred_classes_ave, average='macro'), 
         "precision": precision_score(y_test, pred_classes_ave, average='macro'), 
         "recall": recall_score(y_test, pred_classes_ave, average='macro'),
         "Fbeta": fbeta_score(y_test, pred_classes_ave, beta=beta, average='macro'), 
           }
    metrics = metrics.append(m_ave, ignore_index=True)
    model_predictions[model_name+"_Average"] = pred_classes_ave
    
    
    #We can also do a weighted average, by each individual accuracy
    for k in l1_scores.keys():
        s = l1_scores[k]
        l1_test_predictions[k] = l1_test_predictions[k] * s
    
    prob_weight = l1_test_predictions.sum(axis=0).as_matrix()
    pred_classes_weight = [classifier.classes_[i] for i in np.argmax(prob_weight, axis=1)]
    
    m_weight = {"model": model_name+"_Weighted", 
         "accuracy": accuracy_score(y_test, pred_classes_weight), 
         "F1": f1_score(y_test, pred_classes_weight, average='macro'), 
         "precision": precision_score(y_test, pred_classes_weight, average='macro'), 
         "recall": recall_score(y_test, pred_classes_weight, average='macro'),
         "Fbeta": fbeta_score(y_test, pred_classes_weight, beta=beta, average='macro'), 
           }
    metrics = metrics.append(m_weight, ignore_index=True)
    model_predictions[model_name+"_Weighted"] = pred_classes_weight
    


--- Model: LogisticRegression
- features: AutoColorCorrelogram
- features: BinaryPatternsPyramid
- features: EdgeHistogram
- features: FCTH
- features: FuzzyColorHistogram
- features: FuzzyOpponentHistogram
- features: Gabor
- features: JCD
- features: JpegCoefficientHistogram
- features: LocalBinaryPatterns
- features: LuminanceLayout
- features: OpponentHistogram
- features: PHOG
- features: RotationInvariantLocalBinaryPatterns
- features: ScalableColor
- features: SimpleColorHistogram
- features: Tamura
- features: CEDD
- features: ColorLayout
--- Model: RandomForestClassifier
- features: AutoColorCorrelogram
- features: BinaryPatternsPyramid
- features: EdgeHistogram
- features: FCTH
- features: FuzzyColorHistogram
- features: FuzzyOpponentHistogram
- features: Gabor
- features: JCD
- features: JpegCoefficientHistogram
- features: LocalBinaryPatterns
- features: LuminanceLayout
- features: OpponentHistogram
- features: PHOG
- features: RotationInvariantLocalBinaryPatterns
- feature

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


- features: BinaryPatternsPyramid


/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


- features: EdgeHistogram
- features: FCTH
- features: FuzzyColorHistogram
- features: FuzzyOpponentHistogram
- features: Gabor
- features: JCD
- features: JpegCoefficientHistogram
- features: LocalBinaryPatterns
- features: LuminanceLayout
- features: OpponentHistogram
- features: PHOG
- features: RotationInvariantLocalBinaryPatterns
- features: ScalableColor
- features: SimpleColorHistogram
- features: Tamura
- features: CEDD
- features: ColorLayout
--- Model: KNeighborsClassifier
- features: AutoColorCorrelogram
- features: BinaryPatternsPyramid
- features: EdgeHistogram
- features: FCTH
- features: FuzzyColorHistogram
- features: FuzzyOpponentHistogram
- features: Gabor
- features: JCD
- features: JpegCoefficientHistogram
- features: LocalBinaryPatterns
- features: LuminanceLayout
- features: OpponentHistogram
- features: PHOG
- features: RotationInvariantLocalBinaryPatterns
- features: ScalableColor
- features: SimpleColorHistogram
- features: Tamura
- features: CEDD
- features: C

In [13]:
metrics

model  accuracy        F1  precision  recall  \
0        LogisticRegression_Average    0.8825  0.881501   0.886176  0.8825   
1       LogisticRegression_Weighted    0.8840  0.883295   0.886910  0.8840   
2    RandomForestClassifier_Average    0.8510  0.850282   0.852272  0.8510   
3   RandomForestClassifier_Weighted    0.8570  0.856345   0.857863  0.8570   
4                GaussianNB_Average    0.6040  0.547265   0.717754  0.6040   
5               GaussianNB_Weighted    0.6300  0.581660   0.726474  0.6300   
6             MLPClassifier_Average    0.8925  0.891857   0.894346  0.8925   
7            MLPClassifier_Weighted    0.8965  0.895998   0.898236  0.8965   
8      KNeighborsClassifier_Average    0.8330  0.831004   0.838454  0.8330   
9     KNeighborsClassifier_Weighted    0.8350  0.832879   0.840611  0.8350   
10                      SVC_Average    0.8490  0.847817   0.851720  0.8490   
11                     SVC_Weighted    0.8475  0.846309   0.849565  0.8475   

       Fbeta  
0   0.881449  
1   0.883217  
2   0.850407  
3   0.856496  
4   0.564916  
5   0.595870  
6   0.891884  
7   0.895983  
8   0.831187  
9   0.833100  
10  0.847958  
11  0.846529

In [14]:
metrics.sort_values(by=["accuracy", "F1"], ascending=False).to_csv(dpath+"metrics_late_fusion_Set2.csv", index=False)

In [15]:
model_scores = pd.DataFrame(model_scores)
model_scores

GaussianNB  KNeighborsClassifier  \
AutoColorCorrelogram                    0.525114              0.650082   
BinaryPatternsPyramid                   0.190956              0.447909   
CEDD                                    0.387969              0.660490   
ColorLayout                             0.640433              0.620336   
EdgeHistogram                           0.494456              0.486063   
FCTH                                    0.446045              0.616647   
FuzzyColorHistogram                     0.460469              0.622496   
FuzzyOpponentHistogram                  0.374008              0.582965   
Gabor                                   0.398538              0.399402   
JCD                                     0.470230              0.662514   
JpegCoefficientHistogram                0.436364              0.511649   
LocalBinaryPatterns                     0.208493              0.361903   
LuminanceLayout                         0.242976              0.339926   
OpponentHistogram                       0.412426              0.535442   
PHOG                                    0.388017              0.397857   
RotationInvariantLocalBinaryPatterns    0.246960              0.366911   
ScalableColor                           0.497120              0.553491   
SimpleColorHistogram                    0.423387              0.625544   
Tamura                                  0.348830              0.345950   

                                      LogisticRegression  MLPClassifier  \
AutoColorCorrelogram                            0.709893       0.707901   
BinaryPatternsPyramid                           0.517497       0.477983   
CEDD                                            0.690004       0.702525   
ColorLayout                                     0.645889       0.661850   
EdgeHistogram                                   0.535858       0.539427   
FCTH                                            0.672907       0.676499   
FuzzyColorHistogram                             0.662914       0.653458   
FuzzyOpponentHistogram                          0.558420       0.633337   
Gabor                                           0.358503       0.413434   
JCD                                             0.702045       0.704965   
JpegCoefficientHistogram                        0.602303       0.604863   
LocalBinaryPatterns                             0.403898       0.410338   
LuminanceLayout                                 0.353423       0.373984   
OpponentHistogram                               0.525410       0.587366   
PHOG                                            0.485831       0.482423   
RotationInvariantLocalBinaryPatterns            0.369424       0.387009   
ScalableColor                                   0.552323       0.548419   
SimpleColorHistogram                            0.634345       0.676899   
Tamura                                          0.387361       0.410450   

                                      RandomForestClassifier       SVC  
AutoColorCorrelogram                                0.689932  0.637505  
BinaryPatternsPyramid                               0.445893  0.442396  
CEDD                                                0.675443  0.602519  
ColorLayout                                         0.659514  0.594550  
EdgeHistogram                                       0.426995  0.530938  
FCTH                                                0.634081  0.552091  
FuzzyColorHistogram                                 0.607399  0.558012  
FuzzyOpponentHistogram                              0.600982  0.413970  
Gabor                                               0.383465  0.375528  
JCD                                                 0.682524  0.600006  
JpegCoefficientHistogram                            0.552411  0.409890  
LocalBinaryPatterns                                 0.400426  0.286906  
LuminanceLayout                                     0.356527  0.250528  
OpponentHistogram                 

In [16]:
model_scores.to_csv(dpath+"individual_metrics_late_fusion_Set2.csv")

In [17]:
#Combining predictions of all classifiers
pred_all = pd.DataFrame(model_predictions).mode(axis=1)[0]

{"model": "All Classifiers Mode", 
         "accuracy": accuracy_score(y_test, pred_all), 
         "F1": f1_score(y_test, pred_all, average='macro'), 
         "precision": precision_score(y_test, pred_all, average='macro'), 
         "recall": recall_score(y_test, pred_all, average='macro'),
         "Fbeta": fbeta_score(y_test, pred_all, beta=beta, average='macro'), 
           }

{'F1': 0.88983019317996881,
 'Fbeta': 0.88974806734658707,
 'accuracy': 0.89049999999999996,
 'model': 'All Classifiers Mode',
 'precision': 0.89339374685989181,
 'recall': 0.89049999999999996}